# Объемные признаки

- Скользящие средние и z-score объема
- Volume Profile: Point of Control (POC), Value Area (VA), width, position
- Spike-индексы (резкие всплески)

В этом ноутбуке реализована самая сложная часть - расчёт горизонтального профиля объема по окну.


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✅ Библиотеки загружены")


✅ Библиотеки загружены


## Функции расчета объемных признаков


In [2]:
def volume_ma(volume, window=20):
    """Скользящее среднее объема"""
    return volume.rolling(window=window).mean()


def volume_zscore(volume, window=60):
    """Z-score объема: нормализованное отклонение от среднего"""
    ma = volume.rolling(window=window).mean()
    std = volume.rolling(window=window).std()
    return (volume - ma) / std


def volume_spike(volume, threshold=2.0, window=20):
    """Индикатор всплеска объема: превышает ли объем порог в std"""
    zscore = volume_zscore(volume, window=window)
    return (zscore > threshold).astype(int)


def calculate_volume_profile(df, window=20, num_bins=50):
    """
    Расчет Volume Profile для скользящего окна
    Возвращает: POC (Point of Control), Value Area High, Value Area Low
    """
    poc_list = []
    va_high_list = []
    va_low_list = []
    
    for i in range(len(df)):
        if i < window:
            poc_list.append(np.nan)
            va_high_list.append(np.nan)
            va_low_list.append(np.nan)
            continue
        
        window_data = df.iloc[i-window:i]
        
        # Создаем ценовые уровни (bins)
        price_min = window_data['low'].min()
        price_max = window_data['high'].max()
        bins = np.linspace(price_min, price_max, num_bins)
        
        # Распределяем объем по ценовым уровням
        volume_by_price = np.zeros(len(bins)-1)
        for _, row in window_data.iterrows():
            # Находим bins, которые пересекаются с [low, high] этого бара
            mask = (bins[:-1] >= row['low']) & (bins[1:] <= row['high'])
            volume_by_price[mask] += row['volume'] / mask.sum() if mask.sum() > 0 else 0
        
        # POC: уровень с максимальным объемом
        poc_idx = volume_by_price.argmax()
        poc = (bins[poc_idx] + bins[poc_idx+1]) / 2
        poc_list.append(poc)
        
        # Value Area: 70% объема вокруг POC
        total_volume = volume_by_price.sum()
        target_volume = total_volume * 0.70
        
        # Расширяем от POC вверх и вниз
        va_volume = volume_by_price[poc_idx]
        va_low_idx = poc_idx
        va_high_idx = poc_idx
        
        while va_volume < target_volume and (va_low_idx > 0 or va_high_idx < len(volume_by_price)-1):
            if va_low_idx > 0:
                va_volume += volume_by_price[va_low_idx-1]
                va_low_idx -= 1
            if va_high_idx < len(volume_by_price)-1 and va_volume < target_volume:
                va_volume += volume_by_price[va_high_idx+1]
                va_high_idx += 1
        
        va_low = bins[va_low_idx]
        va_high = bins[va_high_idx+1]
        
        va_high_list.append(va_high)
        va_low_list.append(va_low)
    
    return pd.Series(poc_list, index=df.index), pd.Series(va_high_list, index=df.index), pd.Series(va_low_list, index=df.index)

print("✅ Функции объема загружены")


✅ Функции объема загружены


## Загрузка и расчет признаков


In [5]:
# Получаем текущую рабочую директорию
current_dir = Path.cwd()
print(f"Текущая директория: {current_dir}")

# Строим путь к данным относительно текущей директории
# Если ноутбук в 'ML/02_feature_engineering/', а данные в 'ML/data/processed'
DATA_DIR = current_dir.parent.parent / "ML" / "data" / "processed"
print(f"Путь к данным: {DATA_DIR}")

# Проверяем существование пути
if DATA_DIR.exists():
    print("Директория найдена!")
else:
    print("Директория не найдена, проверьте путь")


ticker = 'SBER'
df = pd.read_parquet(DATA_DIR / f"{ticker}_ohlcv_returns.parquet")

# Расчет объемных признаков
df['volume_ma_20'] = volume_ma(df['volume'], window=20)
df['volume_ma_60'] = volume_ma(df['volume'], window=60)
df['volume_zscore'] = volume_zscore(df['volume'], window=60)
df['volume_spike'] = volume_spike(df['volume'], threshold=2.0, window=20)

# Volume Profile (может занять время)
print("Расчет Volume Profile...")
df['vp_poc'], df['vp_va_high'], df['vp_va_low'] = calculate_volume_profile(df, window=20, num_bins=50)
df['vp_width'] = df['vp_va_high'] - df['vp_va_low']
df['vp_position'] = (df['close'] - df['vp_poc']) / df['vp_width']

print("✅ Объемные признаки рассчитаны")
print(f"\nПример данных:")
print(df[['date', 'close', 'volume', 'volume_ma_20', 'volume_zscore', 'vp_poc', 'volume_spike']].tail())


Текущая директория: e:\Python\VolatilityChecker\MOEXScanner\ML\02_feature_engineering
Путь к данным: e:\Python\VolatilityChecker\MOEXScanner\ML\data\processed
Директория найдена!
Расчет Volume Profile...
✅ Объемные признаки рассчитаны

Пример данных:
                    date   close    volume  volume_ma_20  volume_zscore  \
1296 2025-10-07 23:59:54  293.89  37893983   25338844.70       1.104375   
1297 2025-10-08 23:59:51  281.90  75222569   27794570.65       3.156851   
1298 2025-10-09 23:59:55  289.37  76179258   30032457.00       3.064530   
1299 2025-10-10 23:59:55  285.12  34168796   30364060.35       0.774010   
1300 2025-10-11 15:00:27  284.44   1976492   28516895.45      -1.139398   

          vp_poc  volume_spike  
1296  289.250816             0  
1297  289.250816             1  
1298  289.111020             1  
1299  289.316837             0  
1300  289.320408             0  


## Сохранение результатов


In [ ]:
OUTPUT_DIR = Path('data') / 'features'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

output_path = OUTPUT_DIR / f"{ticker}_volume_features.parquet"
df.to_parquet(output_path, index=False)
print(f"✅ Сохранено: {output_path}")
